In [2]:
!pip install torch==2.2.2
!pip install torch-cluster==1.6.3
!pip install torch-geometric==2.6.1
!pip install torch-scatter==2.1.2
!pip install torch-sparse==0.6.18
!pip install torch-spline-conv==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 78.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━

In [3]:
# !pip install torch-geometric
!pip install ogb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.8 MB/s eta 0:00:00


In [4]:
# !pip install torch-scatter 
# !pip install torch-sparse

# !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
# !pip install torch-geometric
# !pip install -q git+https://github.com/snap-stanford/deepsnap.git


In [5]:
# !pip install -q git+https://github.com/snap-stanford/deepsnap.git

In [6]:
# !pip install torch-cluster

In [7]:
# !pip install --verbose git+https://github.com/pyg-team/pyg-lib.git

In [8]:
# Importing necessary dependencies in order to import our dataset, create our
# GCN models, and evaluate the models

import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU

from torch_geometric.utils import negative_sampling
from torch_geometric.data import Data

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GINConv, GATConv

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from torch_geometric.data.data import DataEdgeAttr, GlobalStorage

from torch_geometric.nn import Node2Vec

import pandas as pd
import shutil, os
import os.path as osp
import numpy as np

#from logger import Logger
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import random


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

Device: cuda


In [10]:
# loaded with transform parameter set as such in order to obtain the adj_t matrix
# required for the GNN layers
# torch.serialization.add_safe_globals([DataEdgeAttr])
# torch.serialization.add_safe_globals([GlobalStorage])
dataset = PygLinkPropPredDataset(name='ogbl-ddi', root ='./', transform=T.ToSparseTensor()) # loading ogb-ddi
print('Task type: {}'.format(dataset.task_type))
graph = dataset[0]
adj_t = graph.adj_t.to(device) # loads all edges in graph into sparse adj_t matrix

Downloaded 0.04 GB: 100%|██████████| 46/46 [00:01<00:00, 23.74it/s]


Extracting ./ddi.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 58.23it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 3264.05it/s]

Saving...
Task type: link prediction



Done!


In [11]:
# getting the train, validation, and test edge splits
split_edge = dataset.get_edge_split()
train_edges = split_edge['train']['edge']
torch.manual_seed(70) # picking random samples to evaluate on
idx = torch.randperm(split_edge['train']['edge'].size(0))
idx = idx[:split_edge['valid']['edge'].size(0)]
split_edge['eval_train'] = {'edge': split_edge['train']['edge'][idx]}

In [12]:
train_edges_node2vec = train_edges.T # transpose to get the right dimension
# Initialize edge-induced subgraph with only train edges (edge-induced subgraph)
data_node2vec = Data(edge_index=train_edges_node2vec)

In [13]:
def save_embedding(model, filepath): # function to save embedding to specified filepath
    torch.save(model.embedding.weight.data.cpu(), filepath)

In [29]:
def load_node2vec(args, data, filepath):
    model = Node2Vec(data.edge_index, args['embedding_dim'], args['walk_length'],
                    args['context_size'], args['walks_per_node'],
                    sparse=True).to(device)
    loader = model.loader(batch_size=args['batch_size'], shuffle=True,
                        num_workers=4)
    optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=args['lr'])

    model.load_state_dict(torch.load(filepath))

    return model

In [22]:
# training function for node2vec using PyG's Node2Vec function
def train_Node2Vec(args, data, filepath):
  model = Node2Vec(data.edge_index, args['embedding_dim'], args['walk_length'],
                    args['context_size'], args['walks_per_node'],
                    sparse=True).to(device)

  loader = model.loader(batch_size=args['batch_size'], shuffle=True,
                        num_workers=4)
  optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=args['lr'])

  model.train()
  for epoch in range(1, args['epochs'] + 1):
      for i, (pos_rw, neg_rw) in enumerate(loader):
          optimizer.zero_grad()
          loss = model.loss(pos_rw.to(device), neg_rw.to(device))
          loss.backward()
          optimizer.step()

          if (i + 1) % args['log_steps'] == 0:
              print(f'Epoch: {epoch:02d}, Step: {i+1:03d}/{len(loader)}, '
                    f'Loss: {loss:.4f}')

          if (i + 1) % 100 == 0:  # Save model every 100 steps.
              save_embedding(model, filepath)
      save_embedding(model, filepath)
  return model

In [23]:
train_node2vec = True # change to true if you'd like to train/retrain embeddings

In [16]:
node2vec_args = {'device':0, 'embedding_dim':256, 'walk_length':40, 'context_size':20, 'walks_per_node':10,
      'batch_size':256, 'lr':0.01, 'epochs':100, 'log_steps':1}

In [17]:
import os
if not os.path.exists('/kaggle/working/training_outputs/'):
    os.makedirs('/kaggle/working/training_outputs/') # directory for saving visualizations and model checkpoints

In [24]:
  # Train Node2Vec on the train edge-induced subgraph
  filepath = '/kaggle/working/training_outputs/train-embedding-256.pt'
  node2vec_args = {'device':0, 'embedding_dim':256, 'walk_length':40, 'context_size':20, 'walks_per_node':10,
      'batch_size':256, 'lr':0.01, 'epochs':100, 'log_steps':1}
  node2vec = train_Node2Vec(node2vec_args, data_node2vec, filepath) # embeddings will save to specified filepath

Epoch: 01, Step: 001/17, Loss: 9.4905
Epoch: 01, Step: 002/17, Loss: 9.2653
Epoch: 01, Step: 003/17, Loss: 9.0823
Epoch: 01, Step: 004/17, Loss: 8.9872
Epoch: 01, Step: 005/17, Loss: 8.8345
Epoch: 01, Step: 006/17, Loss: 8.7417
Epoch: 01, Step: 007/17, Loss: 8.5942
Epoch: 01, Step: 008/17, Loss: 8.5636
Epoch: 01, Step: 009/17, Loss: 8.4319
Epoch: 01, Step: 010/17, Loss: 8.4013
Epoch: 01, Step: 011/17, Loss: 8.2981
Epoch: 01, Step: 012/17, Loss: 8.1747
Epoch: 01, Step: 013/17, Loss: 8.1233
Epoch: 01, Step: 014/17, Loss: 8.0605
Epoch: 01, Step: 015/17, Loss: 8.0044
Epoch: 01, Step: 016/17, Loss: 7.9633
Epoch: 01, Step: 017/17, Loss: 7.8605
Epoch: 02, Step: 001/17, Loss: 7.8060
Epoch: 02, Step: 002/17, Loss: 7.7441
Epoch: 02, Step: 003/17, Loss: 7.6939
Epoch: 02, Step: 004/17, Loss: 7.6498
Epoch: 02, Step: 005/17, Loss: 7.5864
Epoch: 02, Step: 006/17, Loss: 7.5487
Epoch: 02, Step: 007/17, Loss: 7.4917
Epoch: 02, Step: 008/17, Loss: 7.4348
Epoch: 02, Step: 009/17, Loss: 7.3704
Epoch: 02, S

## Upload node2vec to huggingface

In [25]:
node2vec

Node2Vec(4267, 256)

In [28]:
# OR save just the state_dict (recommended)
torch.save(node2vec.state_dict(), "/kaggle/working/training_outputs/node2vec_state_dict.pth")

In [30]:
filepath = "/kaggle/working/training_outputs/node2vec_state_dict.pth"

new_node2vec = load_node2vec(node2vec_args, data_node2vec, filepath)

In [31]:
def compare_models(model_1, model_2):
    for p1, p2 in zip(model_1.parameters(), model_2.parameters()):
        if not torch.equal(p1.data, p2.data):
            return False
    return True

if compare_models(node2vec, new_node2vec):
    print("The saved and loaded models are identical")
else:
    print("The saved and loaded models are different")

The saved and loaded models are identical


In [53]:
from huggingface_hub import HfApi, HfFolder, upload_file, create_repo, hf_hub_download

In [40]:
from huggingface_hub import login

In [48]:
hf_token = os.environ.get("HF_TOKEN")

In [49]:
login()

In [51]:
# # Create a new repo (if it doesn't already exist)
# create_repo("RZoro/ogb_ddi-node2vec-model", repo_type="model", exist_ok=True)

In [52]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/node2vec_state_dict.pth",
    path_in_repo="node2vec_state_dict.pth",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

node2vec_state_dict.pth:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/9c9fc3fb347ba8834403ccf9015a25acb6e96527', commit_message='Upload node2vec_state_dict.pth with huggingface_hub', commit_description='', oid='9c9fc3fb347ba8834403ccf9015a25acb6e96527', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)

In [54]:
file_path = hf_hub_download(
    repo_id="Rzoro/ogb_ddi",  # Replace with your model repo
    filename="node2vec_state_dict.pth",          # Replace with your model file
    repo_type="model"                         # Could be 'dataset' or 'space' too
)

print("Downloaded to:", file_path)

node2vec_state_dict.pth:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

Downloaded to: /root/.cache/huggingface/hub/models--Rzoro--ogb_ddi/snapshots/9c9fc3fb347ba8834403ccf9015a25acb6e96527/node2vec_state_dict.pth


In [55]:
new_node2vec2 = load_node2vec(node2vec_args, data_node2vec, filepath)

In [57]:
new_node2vec2

Node2Vec(4267, 256)

In [56]:
if compare_models(node2vec, new_node2vec2):
    print("The saved and loaded models are identical")
else:
    print("The saved and loaded models are different")

The saved and loaded models are identical


## Train GCN model

In [58]:
# class using PyG's GCNConv layer
class GCN(torch.nn.Module):
    ''' Define GCN network. '''
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        # Execute conv -> relu -> dropout sequence
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

In [59]:
# class in order to predict whether a link exists between two nodes using
# their embeddings, x_i and x_j
class LinkPredictor(torch.nn.Module):
    ''' Neural network which predicts whether a link (interaction) exists between 2 nodes i,j
    given their embeddings x_i, x_j.
    '''
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j # hadamard product
        for lin in self.lins[:-1]: # linear layer -> relu -> dropout
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x) # sigmoid activation outputs probability that a given edge exists for all node pairs

In [60]:
def train(model, predictor, x, adj_t, split_edge, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = split_edge['train']['edge'].to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        # computes the loss for positive edges
        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        # samples negative edges from the graph
        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='dense')

        # computes the loss for negative edges
        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples


@torch.no_grad()
def test(model, predictor, x, adj_t, split_edge, evaluator, batch_size):
    model.eval()
    predictor.eval()

    h = model(x, adj_t)

    pos_train_edge = split_edge['eval_train']['edge'].to(x.device)
    pos_valid_edge = split_edge['valid']['edge'].to(x.device)
    neg_valid_edge = split_edge['valid']['edge_neg'].to(x.device)
    pos_test_edge = split_edge['test']['edge'].to(x.device)
    neg_test_edge = split_edge['test']['edge_neg'].to(x.device)

    # store what the link predictor outputs for each positive and negative
    # edge in order to compute the hits@K
    pos_train_preds = []
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size):
        edge = pos_train_edge[perm].t()
        pos_train_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_train_pred = torch.cat(pos_train_preds, dim=0)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)

    # compute the hits@K for training, validation, and test
    results = {}
    for K in [10, 20]:
        evaluator.K = K
        train_hits = evaluator.eval({
            'y_pred_pos': pos_train_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'Hits@{K}'] = (train_hits, valid_hits, test_hits)

    return results

In [127]:
gnn_args = { # define GNN hyperparams
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'hidden_size': 256,
    'dropout': 0.5,
    'epochs': 100,
    'weight_decay': 1e-5,
    'lr': 0.005,
    'attn_size': 32,
    'num_layers':2,
    'log_steps':1,
    'eval_steps':5,
    'runs':10,
    'batch_size': 1024,
    'attn_heads': 1,

}

In [70]:
pretrained_weight = new_node2vec2.embedding.weight.data.cpu()
# node2vec_emb.weight.shape

In [67]:
dataset.data.num_nodes

4267

In [80]:
# Initialize embedding matrix
node2vec_emb = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
node2vec_emb.weight.data.cpu()

tensor([[-2.5577, -0.5522,  0.7761,  ...,  0.7560,  0.6740, -0.7607],
        [-0.6029,  0.9182,  2.1672,  ..., -0.4306, -1.9059,  1.2720],
        [-0.4639,  0.8636,  2.0954,  ..., -0.6557, -0.2586,  0.6770],
        ...,
        [-0.3463,  0.2970, -0.2306,  ..., -0.4084,  0.1816, -1.1687],
        [-0.5405,  1.5223,  0.3612,  ...,  1.7321,  0.5031, -0.5614],
        [ 0.7525,  2.0423, -0.0928,  ..., -0.1488,  0.3232, -0.3791]])

In [81]:
# get embedding weights from node2vec model
node2vec_emb.weight.data.copy_(pretrained_weight)

tensor([[-8.9925e-01,  3.5224e-01, -1.6614e-01,  ...,  1.6531e-01,
         -5.0935e-01,  2.8803e-01],
        [ 9.0283e-02, -1.7072e-01, -8.8191e-02,  ...,  1.9789e-02,
         -8.3763e-02,  2.4516e-02],
        [-1.1388e-01, -3.9438e-02,  2.2235e-02,  ...,  1.4269e-01,
         -2.7632e-01, -1.1623e-02],
        ...,
        [ 2.6147e-02,  1.0147e-01,  8.9643e-03,  ..., -2.5608e-04,
          1.6373e-02, -7.6566e-02],
        [-1.0164e-02,  5.5890e-02, -1.3590e-02,  ...,  8.7863e-03,
         -8.5676e-02, -4.0881e-02],
        [ 1.0617e-01, -4.3757e-02,  3.1394e-02,  ...,  2.7990e-02,
          7.6857e-02, -3.9015e-02]], device='cuda:0')

Basically we converted a node2vec object to embedding 

In [85]:
# comparing if two embeddings are same
torch.equal(node2vec_emb.weight.data.cpu(), pretrained_weight)

True

In [86]:
node2vec_emb

Embedding(4267, 256)

In [87]:
new_node2vec2

Node2Vec(4267, 256)

In [128]:
def train_model(model, emb, gnn_args, predictor, model_name):
  '''
  Train specified GNN model. Model and embeddings should be initialized.
  Save model after every run.
  '''
  train_hits_arr, val_hits_arr, test_hits_arr = [], [], []

  evaluator = Evaluator(name='ogbl-ddi')
  for run in range(2):
    max_valhits, train_hits_run, test_hits_run = float('-inf'), 0, 0

    torch.nn.init.xavier_uniform_(emb.weight)
    model.reset_parameters()
    predictor.reset_parameters()
    optimizer = torch.optim.Adam(
        list(model.parameters()) + list(emb.parameters()) +
        list(predictor.parameters()), lr=gnn_args['lr'])

    for epoch in range(1, 1 + gnn_args['epochs']):
        loss = train(model, predictor, emb.weight, adj_t, split_edge,
                      optimizer, gnn_args['batch_size'])

        if epoch % gnn_args['eval_steps'] == 0:
            results = test(model, predictor, emb.weight, adj_t, split_edge,
                            evaluator, gnn_args['batch_size'])


            if epoch % gnn_args['log_steps'] == 0:
                for key, result in results.items():
                    train_hits, valid_hits, test_hits = result
                    print(key)
                    print(f'Run: {run + 1:02d}, '
                          f'Epoch: {epoch:02d}, '
                          f'Loss: {loss:.4f}, '
                          f'Train: {100 * train_hits:.2f}%, '
                          f'Valid: {100 * valid_hits:.2f}%, '
                          f'Test: {100 * test_hits:.2f}%')
                print('---')

            # check val-hits@20
            train_hits, valid_hits, test_hits = results['Hits@20']
            if valid_hits >= max_valhits: # if validhits20 is higher than max, save ckpt
              max_valhits = valid_hits
              train_hits_run = train_hits
              test_hits_run = test_hits
              # Save model checkpoint for current run.
              model_path = f"training_outputs/{model_name}.pt"
              emb_path = f'training_outputs/{model_name}_init_emb.pt'
              save_model_ckpt(model, emb, optimizer, predictor, loss, emb_path, model_path)
    train_hits_arr.append(train_hits_run)
    test_hits_arr.append(test_hits_run)
    val_hits_arr.append(max_valhits)


  # Print overall stats arrays for best model based on val hits@20
  print("Val_hits@20: ", val_hits_arr)
  print("Test_hits@20: ", test_hits_arr)
  print("Train_hits@20: ", train_hits_arr)

  # Print best model stats (based on val hits@20)
  val_max = max(val_hits_arr)
  print("Best model val hits@20: ", max(val_hits_arr))
  max_idx = val_hits_arr.index(val_max)
  print('Best model test hits@20: ', test_hits_arr[max_idx])
  print('Best model train hits@20: ', val_hits_arr[max_idx])

  # convert to numpy array
  val_hits_arr = np.array(val_hits_arr)
  test_hits_arr = np.array(test_hits_arr)
  train_hits_arr = np.array(train_hits_arr)

  # Print average stats + variance
  print(f"Average best train hits@20: {np.mean(train_hits_arr)}; var: {np.var(train_hits_arr)}")
  print(f"Average best val hits@20: {np.mean(val_hits_arr)}; var: {np.var(val_hits_arr)}")
  print(f"Average best test hits@20: {np.mean(test_hits_arr)}; var: {np.var(test_hits_arr)}")

In [89]:
def save_model_ckpt(model, emb, optimizer, predictor, loss, emb_path, model_path):
  ''' Save model and embedding checkpoints. '''
  EPOCH = 100
  # Save model params
  torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'predictor_state_dict': predictor.state_dict(),
            'loss': loss,
            }, model_path)
  # Also save initial embedding (just in case)
  torch.save(emb.weight.data.cpu(), emb_path)

In [90]:
def load_model_ckpt(curr_model, model_name, run):
  ''' Load model checkpoint. '''
  evaluator = Evaluator(name='ogbl-ddi')
  model_path = f"training_outputs/{model_name}.pt"
  emb_path = f'training_outputs/{model_name}_init_emb.pt'

  # Load emb (init feature representations)
  pretrained_weight = torch.load(emb_path, map_location='cpu').to(device)
  pretrained_weight = pretrained_weight.cpu().data.numpy()
  emb_after = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
  # Pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
  emb_after.weight.data.copy_(torch.from_numpy(pretrained_weight))

  # Init optimizer and predictor objects
  predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
  optimizer = torch.optim.Adam(
        list(curr_model.parameters()) + list(emb_after.parameters()) +
        list(predictor.parameters()), lr=gnn_args['lr'])


  # Load model, predictor, and optimizer params
  checkpoint = torch.load(model_path)
  curr_model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  predictor.load_state_dict(checkpoint['predictor_state_dict'])
  epoch = checkpoint['epoch']
  loss = checkpoint['loss']


  # Save final embedding representation of all nodes
  h = curr_model(emb_after.weight, adj_t)
  final_emb_path = f'training_outputs/{model_name}_final_emb_{run}.pt'
  torch.save(h, final_emb_path)

  # Evaluate pretrained model
  results = test(curr_model, predictor, emb_after.weight, adj_t, split_edge,
                               evaluator, gnn_args['batch_size'])

  # Print hits stats
  for key, result in results.items():
    print(key)
    train_hits, valid_hits, test_hits = result
    print(f'Train: {100 * train_hits:.2f}%, '
                              f'Valid: {100 * valid_hits:.2f}%, '
                              f'Test: {100 * test_hits:.2f}%')

In [91]:
# TRAIN gcn with random features
model_name = 'gcn_rand_feat'
gcn_model = GCN(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
              gnn_args['num_layers'], gnn_args['dropout']).to(device)
predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
gcn_emb_rand = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
train_model(gcn_model, gcn_emb_rand, gnn_args, predictor, model_name)

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Run: 01, Epoch: 05, Loss: 0.6586, Train: 17.27%, Valid: 15.94%, Test: 12.54%
Hits@20
Run: 01, Epoch: 05, Loss: 0.6586, Train: 18.90%, Valid: 17.70%, Test: 16.43%
---
Hits@10
Run: 01, Epoch: 10, Loss: 0.5010, Train: 3.37%, Valid: 2.72%, Test: 6.64%
Hits@20
Run: 01, Epoch: 10, Loss: 0.5010, Train: 16.45%, Valid: 14.52%, Test: 12.31%
---
Hits@10
Run: 01, Epoch: 15, Loss: 0.4248, Train: 19.47%, Valid: 17.16%, Test: 6.63%
Hits@20
Run: 01, Epoch: 15, Loss: 0.4248, Train: 23.85%, Valid: 21.25%, Test: 11.66%
---
Hits@10
Run: 01, Epoch: 20, Loss: 0.3845, Train: 16.97%, Valid: 14.67%, Test: 12.45%
Hits@20
Run: 01, Epoch: 20, Loss: 0.3845, Train: 23.00%, Valid: 20.31%, Test: 17.31%
---
Hits@10
Run: 01, Epoch: 25, Loss: 0.3585, Train: 15.79%, Valid: 13.52%, Test: 8.45%
Hits@20
Run: 01, Epoch: 25, Loss: 0.3585, Train: 24.29%, Valid: 21.24%, Test: 20.74%
---
Hits@10
Run: 01, Epoch: 30, Loss: 0.3416, Train: 19.82%, Valid: 17.13%, Test: 6.20%
Hits@20
Run: 01, Epoch: 30, Loss: 0.3416, Train: 30

In [94]:
# Loading pretrained GCN (random weight init)
gcn_model = GCN(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)
load_model_ckpt(gcn_model, model_name,0)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Train: 34.38%, Valid: 29.48%, Test: 15.37%
Hits@20
Train: 51.61%, Valid: 45.05%, Test: 30.93%


In [95]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gcn_rand_feat.pt",
    path_in_repo="gcn_rand_feat.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gcn_rand_feat_init_emb.pt",
    path_in_repo="gcn_rand_feat_init_emb.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

gcn_rand_feat.pt:   0%|          | 0.00/11.1M [00:00<?, ?B/s]

gcn_rand_feat_init_emb.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/33d8bbbc71a33eabe6a5212d5c58519f5e192eba', commit_message='Upload gcn_rand_feat_init_emb.pt with huggingface_hub', commit_description='', oid='33d8bbbc71a33eabe6a5212d5c58519f5e192eba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)

In [98]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gcn_rand_feat_final_emb_0.pt",
    path_in_repo="gcn_rand_feat_final_emb_0.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

gcn_rand_feat_final_emb_0.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/70ee025d2758e3c6b4ebe7f807d4febb3a0ddc88', commit_message='Upload gcn_rand_feat_final_emb_0.pt with huggingface_hub', commit_description='', oid='70ee025d2758e3c6b4ebe7f807d4febb3a0ddc88', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)

In [96]:
# Train GCN with Node2Vec features
model_name = 'gcn_node2vec_feat_256'
gcn_model2 = GCN(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)

gcn_predictor2 = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
gcn_emb_node2vec = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
gcn_emb_node2vec.weight.data.copy_(pretrained_weight)
train_model(gcn_model2, gcn_emb_node2vec, gnn_args, gcn_predictor2, model_name)

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Run: 01, Epoch: 05, Loss: 0.6928, Train: 3.43%, Valid: 2.94%, Test: 0.90%
Hits@20
Run: 01, Epoch: 05, Loss: 0.6928, Train: 5.28%, Valid: 4.58%, Test: 2.58%
---
Hits@10
Run: 01, Epoch: 10, Loss: 0.5178, Train: 18.36%, Valid: 16.35%, Test: 12.10%
Hits@20
Run: 01, Epoch: 10, Loss: 0.5178, Train: 23.31%, Valid: 20.89%, Test: 14.44%
---
Hits@10
Run: 01, Epoch: 15, Loss: 0.4410, Train: 10.30%, Valid: 8.80%, Test: 11.78%
Hits@20
Run: 01, Epoch: 15, Loss: 0.4410, Train: 20.63%, Valid: 18.38%, Test: 14.75%
---
Hits@10
Run: 01, Epoch: 20, Loss: 0.3777, Train: 18.04%, Valid: 15.68%, Test: 10.52%
Hits@20
Run: 01, Epoch: 20, Loss: 0.3777, Train: 28.09%, Valid: 24.82%, Test: 18.29%
---
Hits@10
Run: 01, Epoch: 25, Loss: 0.3481, Train: 26.46%, Valid: 23.33%, Test: 8.78%
Hits@20
Run: 01, Epoch: 25, Loss: 0.3481, Train: 36.67%, Valid: 32.69%, Test: 14.79%
---
Hits@10
Run: 01, Epoch: 30, Loss: 0.3320, Train: 13.74%, Valid: 11.71%, Test: 5.94%
Hits@20
Run: 01, Epoch: 30, Loss: 0.3320, Train: 21.19

In [97]:
# Loading GCN trained on Node2Vec Features
gcn_model2 = GCN(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)
load_model_ckpt(gcn_model, model_name,0)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Train: 29.12%, Valid: 24.73%, Test: 25.40%
Hits@20
Train: 44.92%, Valid: 38.92%, Test: 35.20%


In [99]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gcn_node2vec_feat_256.pt",
    path_in_repo="gcn_node2vec_feat_256.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gcn_node2vec_feat_256_final_emb_0.pt",
    path_in_repo="gcn_node2vec_feat_256_final_emb_0.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gcn_node2vec_feat_256_init_emb.pt",
    path_in_repo="gcn_node2vec_feat_256_init_emb.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

gcn_node2vec_feat_256.pt:   0%|          | 0.00/11.1M [00:00<?, ?B/s]

gcn_node2vec_feat_256_final_emb_0.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

gcn_node2vec_feat_256_init_emb.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/415af6201e75ccbc5a176cee7400353399be0017', commit_message='Upload gcn_node2vec_feat_256_init_emb.pt with huggingface_hub', commit_description='', oid='415af6201e75ccbc5a176cee7400353399be0017', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)

In [107]:
# class using PyG's SAGEConv layer
class SAGE(torch.nn.Module):
    ''' Define GCN network. '''
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
      # Execute conv -> relu -> dropout sequence
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

In [108]:
# TRAIN GRAPHSAGE with random features
model_name = 'sage_rand_feat'
sage_model = SAGE(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
              gnn_args['num_layers'], gnn_args['dropout']).to(device)
sage_predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
sage_emb_rand = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
train_model(sage_model, sage_emb_rand, gnn_args, sage_predictor, model_name)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Run: 01, Epoch: 05, Loss: 0.5733, Train: 8.38%, Valid: 7.35%, Test: 5.86%
Hits@20
Run: 01, Epoch: 05, Loss: 0.5733, Train: 12.85%, Valid: 11.38%, Test: 9.00%
---
Hits@10
Run: 01, Epoch: 10, Loss: 0.4084, Train: 28.46%, Valid: 25.33%, Test: 9.78%
Hits@20
Run: 01, Epoch: 10, Loss: 0.4084, Train: 32.24%, Valid: 28.78%, Test: 16.75%
---
Hits@10
Run: 01, Epoch: 15, Loss: 0.3221, Train: 32.78%, Valid: 28.39%, Test: 5.23%
Hits@20
Run: 01, Epoch: 15, Loss: 0.3221, Train: 39.36%, Valid: 34.41%, Test: 12.56%
---
Hits@10
Run: 01, Epoch: 20, Loss: 0.2858, Train: 38.63%, Valid: 33.21%, Test: 5.82%
Hits@20
Run: 01, Epoch: 20, Loss: 0.2858, Train: 43.74%, Valid: 37.96%, Test: 11.02%
---
Hits@10
Run: 01, Epoch: 25, Loss: 0.2560, Train: 42.11%, Valid: 36.05%, Test: 10.33%
Hits@20
Run: 01, Epoch: 25, Loss: 0.2560, Train: 48.19%, Valid: 41.62%, Test: 21.04%
---
Hits@10
Run: 01, Epoch: 30, Loss: 0.2379, Train: 51.01%, Valid: 43.95%, Test: 11.48%
Hits@20
Run: 01, Epoch: 30, Loss: 0.2379, Train: 56.

In [109]:
# Loading pretrained graphsage (random weight init) [model with best tests hits@20]
sage_model = SAGE(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)


load_model_ckpt(sage_model, model_name,0)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Train: 62.49%, Valid: 52.65%, Test: 46.85%
Hits@20
Train: 72.17%, Valid: 62.21%, Test: 67.13%


In [110]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/sage_rand_feat.pt",
    path_in_repo="sage_rand_feat.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/sage_rand_feat_final_emb_0.pt",
    path_in_repo="sage_rand_feat_final_emb_0.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/sage_rand_feat_init_emb.pt",
    path_in_repo="sage_rand_feat_init_emb.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

sage_rand_feat.pt:   0%|          | 0.00/12.7M [00:00<?, ?B/s]

sage_rand_feat_final_emb_0.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

sage_rand_feat_init_emb.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/fc532e539b0b627237324971215810885c99710a', commit_message='Upload sage_rand_feat_init_emb.pt with huggingface_hub', commit_description='', oid='fc532e539b0b627237324971215810885c99710a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)

In [112]:
# Train GraphSAGE with node2vec = 256 features
model_name = 'sage_node2vec_feat_256'
sage_model2 = SAGE(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)

sage_predictor2 = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
sage_emb_node2vec = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
sage_emb_node2vec.weight.data.copy_(pretrained_weight)
train_model(sage_model2, sage_emb_node2vec, gnn_args, sage_predictor2, 'sage_node2vec_feat_256')

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Run: 01, Epoch: 05, Loss: 0.5644, Train: 16.31%, Valid: 14.86%, Test: 3.31%
Hits@20
Run: 01, Epoch: 05, Loss: 0.5644, Train: 20.05%, Valid: 18.39%, Test: 8.41%
---
Hits@10
Run: 01, Epoch: 10, Loss: 0.3869, Train: 30.05%, Valid: 26.28%, Test: 14.28%
Hits@20
Run: 01, Epoch: 10, Loss: 0.3869, Train: 37.73%, Valid: 33.53%, Test: 23.35%
---
Hits@10
Run: 01, Epoch: 15, Loss: 0.3111, Train: 29.08%, Valid: 24.77%, Test: 12.80%
Hits@20
Run: 01, Epoch: 15, Loss: 0.3111, Train: 37.67%, Valid: 32.40%, Test: 18.07%
---
Hits@10
Run: 01, Epoch: 20, Loss: 0.2733, Train: 48.02%, Valid: 41.63%, Test: 18.20%
Hits@20
Run: 01, Epoch: 20, Loss: 0.2733, Train: 51.08%, Valid: 44.61%, Test: 23.13%
---
Hits@10
Run: 01, Epoch: 25, Loss: 0.2473, Train: 42.33%, Valid: 35.88%, Test: 11.22%
Hits@20
Run: 01, Epoch: 25, Loss: 0.2473, Train: 52.28%, Valid: 45.01%, Test: 20.93%
---
Hits@10
Run: 01, Epoch: 30, Loss: 0.2309, Train: 46.44%, Valid: 39.12%, Test: 14.83%
Hits@20
Run: 01, Epoch: 30, Loss: 0.2309, Train

In [113]:
# Loading pretrained graphsage (random weight init) [model with best tests hits@20]
sage_model2 = SAGE(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)


load_model_ckpt(sage_model2, model_name,0)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Train: 69.30%, Valid: 59.60%, Test: 25.95%
Hits@20
Train: 73.58%, Valid: 63.87%, Test: 49.52%


In [114]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/sage_node2vec_feat_256.pt",
    path_in_repo="sage_node2vec_feat_256.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/sage_node2vec_feat_256_final_emb_0.pt",
    path_in_repo="sage_node2vec_feat_256_final_emb_0.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/sage_node2vec_feat_256_init_emb.pt",
    path_in_repo="sage_node2vec_feat_256_init_emb.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

sage_node2vec_feat_256.pt:   0%|          | 0.00/12.7M [00:00<?, ?B/s]

sage_node2vec_feat_256_final_emb_0.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

sage_node2vec_feat_256_init_emb.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/3298fc3ecf1541707e10cdeb9fa01874ba932438', commit_message='Upload sage_node2vec_feat_256_init_emb.pt with huggingface_hub', commit_description='', oid='3298fc3ecf1541707e10cdeb9fa01874ba932438', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)

In [115]:
# class using PyG's GINConv layer
class GIN(torch.nn.Module):
    ''' Define graph isomorphic network. '''
    def __init__(self, in_channels, dim, out_channels, num_layers,
                 dropout):
        super().__init__()
         # # Initialize 2 GINConv layers (following num_layers=2 in model-agnostic hyperparams as specified in the blogpost)
        self.conv1 = GINConv(
            Sequential(Linear(in_channels, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU())) # GINConv takes a neural network as input

        self.conv2 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.dropout = dropout

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()


    def forward(self, x, adj_t):
       # Execute conv -> relu -> dropout sequence
        x = self.conv1(x, adj_t)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, adj_t)
        return x

In [116]:
# Train GIN with random features
model_name = "gin_rand_feat"
gin_model = GIN(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)

gin_predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
gin_emb_rand = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
train_model(gin_model, gin_emb_rand, gnn_args, gin_predictor, model_name)

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Run: 01, Epoch: 05, Loss: 0.5476, Train: 2.48%, Valid: 2.08%, Test: 2.59%
Hits@20
Run: 01, Epoch: 05, Loss: 0.5476, Train: 5.12%, Valid: 4.27%, Test: 3.19%
---
Hits@10
Run: 01, Epoch: 10, Loss: 0.4828, Train: 17.83%, Valid: 15.55%, Test: 9.29%
Hits@20
Run: 01, Epoch: 10, Loss: 0.4828, Train: 25.39%, Valid: 22.50%, Test: 16.27%
---
Hits@10
Run: 01, Epoch: 15, Loss: 0.4137, Train: 20.83%, Valid: 18.24%, Test: 12.02%
Hits@20
Run: 01, Epoch: 15, Loss: 0.4137, Train: 24.90%, Valid: 21.69%, Test: 23.88%
---
Hits@10
Run: 01, Epoch: 20, Loss: 0.3714, Train: 25.54%, Valid: 22.47%, Test: 25.96%
Hits@20
Run: 01, Epoch: 20, Loss: 0.3714, Train: 35.17%, Valid: 31.09%, Test: 33.27%
---
Hits@10
Run: 01, Epoch: 25, Loss: 0.3431, Train: 9.52%, Valid: 8.05%, Test: 4.29%
Hits@20
Run: 01, Epoch: 25, Loss: 0.3431, Train: 14.28%, Valid: 12.28%, Test: 5.87%
---
Hits@10
Run: 01, Epoch: 30, Loss: 0.3198, Train: 20.10%, Valid: 17.43%, Test: 17.89%
Hits@20
Run: 01, Epoch: 30, Loss: 0.3198, Train: 28.06%,

In [117]:
# Load GIN with random features
gin_model = GIN(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)
load_model_ckpt(gin_model, model_name,0)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Train: 32.97%, Valid: 28.20%, Test: 37.31%
Hits@20
Train: 46.96%, Valid: 40.57%, Test: 48.98%


In [118]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gin_rand_feat.pt",
    path_in_repo="gin_rand_feat.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gin_rand_feat_final_emb_0.pt",
    path_in_repo="gin_rand_feat_final_emb_0.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gin_rand_feat_init_emb.pt",
    path_in_repo="gin_rand_feat_init_emb.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

gin_rand_feat.pt:   0%|          | 0.00/12.7M [00:00<?, ?B/s]

gin_rand_feat_final_emb_0.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

gin_rand_feat_init_emb.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/7d8f494bde487f391c7290bffc32922bdf47c25f', commit_message='Upload gin_rand_feat_init_emb.pt with huggingface_hub', commit_description='', oid='7d8f494bde487f391c7290bffc32922bdf47c25f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)

In [119]:
# Train GIN with random features
model_name = "gin_node2vec_feat_256"
gin_model2 = GIN(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)

gin_predictor2 = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
gin_emb_node2vec = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
gin_emb_node2vec.weight.data.copy_(pretrained_weight)
train_model(gin_model2, gin_emb_node2vec, gnn_args, gin_predictor2, model_name)

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Run: 01, Epoch: 05, Loss: 0.5503, Train: 12.04%, Valid: 10.52%, Test: 2.53%
Hits@20
Run: 01, Epoch: 05, Loss: 0.5503, Train: 15.44%, Valid: 13.51%, Test: 3.09%
---
Hits@10
Run: 01, Epoch: 10, Loss: 0.4696, Train: 11.89%, Valid: 10.27%, Test: 7.60%
Hits@20
Run: 01, Epoch: 10, Loss: 0.4696, Train: 15.87%, Valid: 13.85%, Test: 10.18%
---
Hits@10
Run: 01, Epoch: 15, Loss: 0.4036, Train: 15.58%, Valid: 13.39%, Test: 7.53%
Hits@20
Run: 01, Epoch: 15, Loss: 0.4036, Train: 22.08%, Valid: 19.30%, Test: 11.30%
---
Hits@10
Run: 01, Epoch: 20, Loss: 0.3693, Train: 11.24%, Valid: 9.67%, Test: 20.48%
Hits@20
Run: 01, Epoch: 20, Loss: 0.3693, Train: 15.69%, Valid: 13.68%, Test: 24.67%
---
Hits@10
Run: 01, Epoch: 25, Loss: 0.3399, Train: 28.28%, Valid: 25.04%, Test: 9.08%
Hits@20
Run: 01, Epoch: 25, Loss: 0.3399, Train: 34.22%, Valid: 30.49%, Test: 17.62%
---
Hits@10
Run: 01, Epoch: 30, Loss: 0.3153, Train: 27.32%, Valid: 23.75%, Test: 21.96%
Hits@20
Run: 01, Epoch: 30, Loss: 0.3153, Train: 43

In [120]:
# Load GIN with random features
gin_model2 = GIN(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],
             gnn_args['num_layers'], gnn_args['dropout']).to(device)
load_model_ckpt(gin_model2, model_name,0)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Train: 35.26%, Valid: 29.80%, Test: 30.34%
Hits@20
Train: 49.21%, Valid: 42.34%, Test: 46.49%


In [121]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gin_node2vec_feat_256.pt",
    path_in_repo="gin_node2vec_feat_256.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gin_node2vec_feat_256_final_emb_0.pt",
    path_in_repo="gin_node2vec_feat_256_final_emb_0.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gin_node2vec_feat_256_init_emb.pt",
    path_in_repo="gin_node2vec_feat_256_init_emb.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

gin_node2vec_feat_256.pt:   0%|          | 0.00/12.7M [00:00<?, ?B/s]

gin_node2vec_feat_256_final_emb_0.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

gin_node2vec_feat_256_init_emb.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/73c5c4f0b4718e53e29226ecf164d2aeddd832a6', commit_message='Upload gin_node2vec_feat_256_init_emb.pt with huggingface_hub', commit_description='', oid='73c5c4f0b4718e53e29226ecf164d2aeddd832a6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)

# Graph Attention Network

In [102]:
from torch_geometric.nn import GATConv
# class using PyG's GATConv layer
class GAN(torch.nn.Module):
    ''' Define graph isomorphic network. '''
    def __init__(self, in_channels, dim, out_channels, attn_size,
                 num_layers, dropout):
        super(GAN, self).__init__()
        self.in_head = attn_size
        self.out_head = 1
        self.dropout = dropout
        # Initialize 2 GATConv layers
        self.conv1 = GATConv(in_channels, dim, heads=self.in_head,
                             dropout=self.dropout)
        self.conv2 = GATConv(dim*self.in_head, out_channels, concat=False,
                             heads=self.out_head, dropout=self.dropout)


    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()


    def forward(self, x, adj_t):
        # Execute conv -> relu -> dropout sequence
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv1(x, adj_t)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, adj_t)
        return x

In [132]:
class GAT(torch.nn.Module):
    '''Define GAT network using PyG's GATConv layer.'''
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                dropout=0.5, heads=1):
        super(GAT, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout))
        
        for _ in range(num_layers - 2):
            self.convs.append(
                GATConv(hidden_channels * heads, hidden_channels, heads=heads, dropout=dropout)
            )
        
        self.convs.append(GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, edge_index)
        return x

In [137]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True "

In [138]:
# TRAIN gcn with random features
model_name = 'gan_rand_feat'
gan_model = GAT(in_channels = gnn_args['hidden_size'],hidden_channels = 32,out_channels = gnn_args['hidden_size'],
              num_layers = gnn_args['num_layers'], dropout = gnn_args['dropout'], heads = 1).to(device)
gan_predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
gan_emb_rand = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
train_model(gan_model, gan_emb_rand, gnn_args, gan_predictor, model_name)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Run: 01, Epoch: 05, Loss: 0.3521, Train: 25.70%, Valid: 23.26%, Test: 0.96%
Hits@20
Run: 01, Epoch: 05, Loss: 0.3521, Train: 35.73%, Valid: 32.73%, Test: 10.85%
---


KeyboardInterrupt: 

In [124]:
# # TRAIN gcn with random features
# model_name = 'gan_rand_feat'
# gan_model = GAN(gnn_args['hidden_size'], gnn_args['hidden_size'], gnn_args['hidden_size'],gnn_args['attn_size'],
#               gnn_args['num_layers'], gnn_args['dropout']).to(device)
# gan_predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
#                           gnn_args['num_layers'], gnn_args['dropout']).to(device)
# gan_emb_rand = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
# train_model(gan_model, gan_emb_rand, gnn_args, gan_predictor, model_name)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


OutOfMemoryError: CUDA out of memory. Tried to allocate 65.31 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.84 GiB is free. Process 2558 has 11.89 GiB memory in use. Of the allocated memory 11.01 GiB is allocated by PyTorch, and 772.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)